In [7]:
from langgraph.graph import StateGraph, END, START
from langchain_google_genai import ChatGoogleGenerativeAI
from typing import TypedDict
from dotenv import load_dotenv
import os
from langchain_core.prompts import PromptTemplate

In [8]:
load_dotenv()

True

In [9]:
model = ChatGoogleGenerativeAI(
    model = 'gemini-2.5-flash',
    google_api_key = os.getenv('GEMINI_API_KEY')
)


In [10]:
class prompt_chain(TypedDict):
    topic: str
    outine: str
    blog: str

In [11]:
def get_outline(state : prompt_chain) -> prompt_chain:

    #fetch title
    topic = state['topic']

    prompt = f"Generate an outline for a blog based on this topic: {topic}"
    outline = model.invoke(prompt).content

    state['outine'] = outline

    return state

In [17]:
def get_final_blog(state : prompt_chain) -> prompt_chain:

    outline = state['outine']
    topic = state['topic']

    final_prompt = f"Write a detailed blog on the title - {topic} based on the outline -\n  {outline}"

    content = model.invoke(final_prompt).content
    state['blog'] = content

    return state

In [18]:
graph = StateGraph(prompt_chain)

graph.add_node('get_outline', get_outline)
graph.add_node('get_final_blog', get_final_blog)

graph.add_edge(START, 'get_outline')
graph.add_edge('get_outline', 'get_final_blog')
graph.add_edge('get_final_blog',END)

workflow = graph.compile()

In [19]:
#Executing the whole thing

initial_state = {'topic' : 'Ratan Tata'}
final_state = workflow.invoke(initial_state)
print(final_state)

{'topic': 'Ratan Tata', 'outine': 'Here\'s a comprehensive outline for a blog post about Ratan Tata, designed to be engaging, informative, and inspiring.\n\n---\n\n## Blog Post Outline: Ratan Tata - The Architect of Modern India\'s Global Ambition\n\n**Target Audience:** General public, aspiring entrepreneurs, business students, anyone interested in leadership, Indian business, or philanthropy.\n\n**Tone:** Respectful, admiring, informative, slightly inspirational.\n\n**SEO Keywords:** Ratan Tata, Tata Group, Indian business, leadership, philanthropy, JLR, Nano, Corus, Tata Trusts, corporate ethics.\n\n---\n\n### I. Catchy Title Options:\n*   Ratan Tata: A Legacy of Vision, Values, and Global Transformation\n*   The Quiet Architect: How Ratan Tata Built India\'s Global Business Empire\n*   Beyond Billions: The Enduring Impact of Ratan Tata on Business and Society\n*   Ratan Tata: More Than a Businessman, a Nation Builder\n\n### II. Introduction (Approx. 150-200 words)\n*   **A. Hook:**